In [1]:
import pandas as pd
from snv import *
df = pd.read_csv('simulation_data_cluster_5_region_6_read_depth_100_replica_2.tsv', sep = '\t')
snv_s = []
for i in range(10):
    start = i * 6
    end = i * 6 + 6
    mat = df.iloc[start : end, ::]
    snv = SNV()
    snv.initialize_by_matrix(mat)
    snv.get_likelihood()
    snv_s.append(snv)
snvs1 = snvs()
snvs1.initialize_by_list(snv_s)
snvs1.set_omega(150)

In [2]:
df.iloc[10 , ]

mutation                    chr6:55104047
region                                 R5
ref_counts                             79
alt_counts                             16
normal_cn                               2
major_cn                                3
minor_cn                                1
tumour_purity                       0.424
ccf                                  0.45
cluster                                 4
SNV-specific copy number              1.0
Name: 10, dtype: object

In [3]:
from snv2 import *
get_b_mat(df)

array([[1., 1., 1., 1., 1., 1.],
       [1., 3., 1., 2., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       ...,
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.]])

In [20]:
import numpy as np
from scipy.optimize import minimize
n = 0
while(n < 10):
    n = n + 1
    for i in range(snvs1.num_snvs):
        start = i * snvs1.num_regions
        end = (i + 1) * snvs1.num_regions
        p0 = snvs1.p[start: end]
        res = minimize(snvs1.obj_func_p2(i), p0, method='nelder-mead',
               options={'xatol': 1e-2, 'disp': True})
        snvs1.set_p_single_snv(res.x, i)

    for i in range(len(snvs1.combination)):
        v0 = snvs1.get_v(i)
        res = minimize(snvs1.obj_func_v(i), v0, method='nelder-mead',
               options={'xatol': 1e-2, 'disp': True})
        snvs1.set_v(res.x, i)
        snvs1.update_y(i)


Optimization terminated successfully.
         Current function value: 247.088130
         Iterations: 280
         Function evaluations: 438
Optimization terminated successfully.
         Current function value: 268.998141
         Iterations: 404
         Function evaluations: 623
Optimization terminated successfully.
         Current function value: 170.830321
         Iterations: 249
         Function evaluations: 389
Optimization terminated successfully.
         Current function value: 289.683068
         Iterations: 309
         Function evaluations: 489
Optimization terminated successfully.
         Current function value: 350.483414
         Iterations: 413
         Function evaluations: 633
Optimization terminated successfully.
         Current function value: 274.139872
         Iterations: 383
         Function evaluations: 592
Optimization terminated successfully.
         Current function value: 208.470886
         Iterations: 257
         Function evaluations: 396
Optimi

/Users/yuding/Dropbox/MDA/Multi-region CliPP/pythonProject/snv.py:200: RuntimeWarning: invalid value encountered in log
  likelihood = np.sum(self.reads * np.log(prop) + (self.total_reads - self.reads) * np.log(1 - prop))


Optimization terminated successfully.
         Current function value: 40110.981565
         Iterations: 519
         Function evaluations: 787
Optimization terminated successfully.
         Current function value: 37261.332972
         Iterations: 475
         Function evaluations: 741
Optimization terminated successfully.
         Current function value: 33724.968711
         Iterations: 463
         Function evaluations: 716
Optimization terminated successfully.
         Current function value: 28732.011963
         Iterations: 555
         Function evaluations: 866
Optimization terminated successfully.
         Current function value: 27864.049392
         Iterations: 382
         Function evaluations: 592
Optimization terminated successfully.
         Current function value: 32735.182537
         Iterations: 453
         Function evaluations: 700


/Users/yuding/Dropbox/MDA/Multi-region CliPP/pythonProject/snv.py:200: RuntimeWarning: divide by zero encountered in log
  likelihood = np.sum(self.reads * np.log(prop) + (self.total_reads - self.reads) * np.log(1 - prop))
/Users/yuding/Dropbox/MDA/Multi-region CliPP/pythonProject/snv.py:200: RuntimeWarning: invalid value encountered in multiply
  likelihood = np.sum(self.reads * np.log(prop) + (self.total_reads - self.reads) * np.log(1 - prop))


Optimization terminated successfully.
         Current function value: 4297.049189
         Iterations: 756
         Function evaluations: 1199
Optimization terminated successfully.
         Current function value: 416.056055
         Iterations: 142
         Function evaluations: 222
Optimization terminated successfully.
         Current function value: 235.737736
         Iterations: 94
         Function evaluations: 152
Optimization terminated successfully.
         Current function value: 570.933197
         Iterations: 117
         Function evaluations: 185
Optimization terminated successfully.
         Current function value: 1158.773952
         Iterations: 140
         Function evaluations: 213
Optimization terminated successfully.
         Current function value: 1392.863342
         Iterations: 205
         Function evaluations: 319
Optimization terminated successfully.
         Current function value: 4811.520500
         Iterations: 130
         Function evaluations: 208
Op

/var/folders/w9/kx7wk6017tl5yqsllw_f5hph0000gn/T/ipykernel_98743/547830631.py:10: RuntimeWarning: Maximum number of function evaluations has been exceeded.
  res = minimize(snvs1.obj_func_p2(i), p0, method='nelder-mead',


In [21]:
snvs1.dis_cluster()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [7]:
import pandas as pd
from snv2 import *

df = pd.read_csv('simulation_data_cluster_5_region_6_read_depth_100_replica_2.tsv', sep = '\t')
df = df.iloc[0: 120, :]
n = 20
m = 6

rho = 5
gamma = 20000000
omega = 500
max_iteration = 50

res = ADMM(df, rho, gamma, omega, n, m, max_iteration)

/Users/yuding/Dropbox/MDA/Multi-region CliPP/pythonProject/snv2.py:118: RuntimeWarning: invalid value encountered in log
  res = -1 * np.sum(read_vec * np.log(prop) + (total_read_vec - read_vec) * np.log(1 - prop))
/Users/yuding/Dropbox/MDA/Multi-region CliPP/pythonProject/snv2.py:118: RuntimeWarning: divide by zero encountered in log
  res = -1 * np.sum(read_vec * np.log(prop) + (total_read_vec - read_vec) * np.log(1 - prop))
/Users/yuding/Dropbox/MDA/Multi-region CliPP/pythonProject/snv2.py:118: RuntimeWarning: invalid value encountered in multiply
  res = -1 * np.sum(read_vec * np.log(prop) + (total_read_vec - read_vec) * np.log(1 - prop))


{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 7, 9: 7, 10: 7, 11: 7, 12: 7, 13: 7, 14: 7, 15: 7, 16: 7, 17: 7, 18: 18, 19: 19}


In [4]:
[df.iloc[0:(6*n), :].cluster[6 * i] for i in range(n)]

[4, 4, 3, 1, 0, 1, 1, 1, 0, 1, 1, 1, 3, 4, 1, 1, 1, 1, 1, 1]

In [1]:
import pandas as pd
from snv2 import *

df = pd.read_csv('simulation_data_cluster_5_region_6_read_depth_100_replica_2.tsv', sep = '\t')
df = df.iloc[0: 120, :]
n = 20
m = 6

sets = {i for i in range(n)}
combinations_2 = list(itertools.combinations(sets, 2))
dic1 = {}
dic2 = {}
index = 0
for i in range(len(combinations_2)):
    combination = combinations_2[i]
    dic1[combination] = index
    dic2[index] = combination
    index = index + 1
pairs_mapping = dic1
pairs_mapping_inverse = dic2

b_mat = get_b_mat(df)
tumor_cn_mat = get_tumor_cn_mat(df)
normal_cn_mat = get_normal_cn_mat(df)
purity_mat = get_purity_mat(df)
read_mat = get_read_mat(df)
total_rad_mat = get_total_read_mat(df)

rho = 10
gamma = 200
omega = 5

In [2]:
p = np.zeros([n*m])
v = np.ones([len(combinations_2) * m])
y = np.ones([len(combinations_2) * m])

from scipy.optimize import minimize
k = 0
while(k < 50):
    k = k + 1
    for i in range(n):
        start = i * m
        end = (i + 1) * m
        p0 = p[start: end]
        res = minimize(get_obj_one_snv_p(i, p, v, y, rho, pairs_mapping, b_mat[i, :], tumor_cn_mat[i, :], normal_cn_mat[i, :], purity_mat[i, :], read_mat[i, :], total_rad_mat[i, :], n, m), 
                       p0, 
                       method='nelder-mead',
                       options={'xatol': 1e-2, 'disp': True})
        p[start: end] = res.x

    for i in range(len(combinations_2)):
        pair = combinations_2[i]
        index_v = pairs_mapping[pair]
        start_v = index_v * m
        end_v = (index_v + 1) * m
        v0 = v[start_v: end_v]
        v[start_v: end_v] = update_v(index_v, pairs_mapping_inverse, p, y, m, rho, omega, gamma)
        y[start_v: end_v] = update_y(y[start_v: end_v], v[start_v: end_v], i, pairs_mapping_inverse, p, m, rho)


Optimization terminated successfully.
         Current function value: 378.816175
         Iterations: 629
         Function evaluations: 967
Optimization terminated successfully.
         Current function value: 416.463334
         Iterations: 343
         Function evaluations: 551
Optimization terminated successfully.
         Current function value: 240.980304
         Iterations: 583
         Function evaluations: 910
Optimization terminated successfully.
         Current function value: 430.870926
         Iterations: 521
         Function evaluations: 801
Optimization terminated successfully.
         Current function value: 482.248547
         Iterations: 392
         Function evaluations: 624
Optimization terminated successfully.
         Current function value: 541.226291
         Iterations: 475
         Function evaluations: 735
Optimization terminated successfully.
         Current function value: 390.364806
         Iterations: 625
         Function evaluations: 968
Optimi

/var/folders/w9/kx7wk6017tl5yqsllw_f5hph0000gn/T/ipykernel_3333/1379461097.py:13: RuntimeWarning: Maximum number of function evaluations has been exceeded.
  res = minimize(get_obj_one_snv_p(i, p, v, y, rho, pairs_mapping, b_mat[i, :], tumor_cn_mat[i, :], normal_cn_mat[i, :], purity_mat[i, :], read_mat[i, :], total_rad_mat[i, :], n, m),


Optimization terminated successfully.
         Current function value: 887.791575
         Iterations: 334
         Function evaluations: 504
Optimization terminated successfully.
         Current function value: 906.006993
         Iterations: 351
         Function evaluations: 549
Optimization terminated successfully.
         Current function value: 667.828925
         Iterations: 274
         Function evaluations: 430
Optimization terminated successfully.
         Current function value: 778.054463
         Iterations: 345
         Function evaluations: 531
Optimization terminated successfully.
         Current function value: 795.603622
         Iterations: 265
         Function evaluations: 420
Optimization terminated successfully.
         Current function value: 844.585318
         Iterations: 394
         Function evaluations: 613
Optimization terminated successfully.
         Current function value: 617.399691
         Iterations: 267
         Function evaluations: 405
Optimi

/Users/yuding/Dropbox/MDA/Multi-region CliPP/pythonProject/snv2.py:118: RuntimeWarning: invalid value encountered in log
  res = -1 * np.sum(read_vec * np.log(prop) + (total_read_vec - read_vec) * np.log(1 - prop))


Optimization terminated successfully.
         Current function value: 2179.094771
         Iterations: 256
         Function evaluations: 396
Optimization terminated successfully.
         Current function value: 1882.466220
         Iterations: 242
         Function evaluations: 377
Optimization terminated successfully.
         Current function value: 1963.277252
         Iterations: 284
         Function evaluations: 449
Optimization terminated successfully.
         Current function value: 1890.841153
         Iterations: 273
         Function evaluations: 420
Optimization terminated successfully.
         Current function value: 1655.898086
         Iterations: 264
         Function evaluations: 424
Optimization terminated successfully.
         Current function value: 1701.378572
         Iterations: 298
         Function evaluations: 461
Optimization terminated successfully.
         Current function value: 1622.451851
         Iterations: 276
         Function evaluations: 428

In [3]:
dis_cluster(v, n, m, combinations_2, pairs_mapping)

{0: 0,
 1: 0,
 2: 2,
 3: 2,
 4: 2,
 5: 2,
 6: 2,
 7: 2,
 8: 2,
 9: 2,
 10: 2,
 11: 2,
 12: 12,
 13: 13,
 14: 14,
 15: 15,
 16: 16,
 17: 17,
 18: 18,
 19: 19}

In [8]:
[df.iloc[0:120, :].cluster[6 * i] for i in range(20)]

[4, 4, 3, 1, 0, 1, 1, 1, 0, 1, 1, 1, 3, 4, 1, 1, 1, 1, 1, 1]

In [10]:
import numpy as np
from scipy.optimize import minimize
def rosen(x):
    """The Rosenbrock function"""
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
res = minimize(rosen, x0, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 339
         Function evaluations: 571


In [15]:
def func1(b):
    def func2(a):
        return b * a ** 2 + 100
    return func2

res = minimize(func1(10), -100, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})

Optimization terminated successfully.
         Current function value: 100.000000
         Iterations: 37
         Function evaluations: 75


In [23]:
ret = []
ret.extend([1,2, 3])
np.array(ret)

array([1, 2, 3])

In [27]:
np.linalg.norm([[1, 1], [1, 2], [3, 4]])**2

32.00000000000001

In [7]:
aa = np.kron([1,-1,0], np.diag([1,1,1])) + np.kron([0, 1,-1], np.diag([1,1,1])) + np.kron([1,0,-1], np.diag([1,1,1]))
aa.shape

(3, 9)

In [8]:
np.ones(3)

array([1., 1., 1.])

In [9]:
arr[1]

NameError: name 'arr' is not defined

In [ ]:
aa = df.tumour_purity[1] * (df.major_cn[1] + df.minor_cn[1]) + (1 - df.tumour_purity[1]) * df.normal_cn[1]
aa

In [10]:
np.round(df.alt_counts[1] / (df.ref_counts[1] +df.alt_counts[1]) * aa / df.tumour_purity[1])

array([[ 1.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0., -1.]])

In [11]:
b = np.min([np.round(df.alt_counts[1] / (df.ref_counts[1] +df.alt_counts[1]) * aa / df.tumour_purity[1]), df.major_cn[1]])
b

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [12]:
from scipy.stats import norm

In [13]:
norm.ppf(0.001)

-3.090232306167813

In [15]:
a1 = a_mat_generator(0, 1, 3, 3)
a2 = a_mat_generator(1, 2, 3, 3)
a3 = a_mat_generator(0, 2, 3, 3)

In [18]:
np.matmul(a1.T, a1) + np.matmul(a2.T, a2) + np.matmul(a3.T, a3)

array([[ 2.,  0.,  0., -1.,  0.,  0., -1.,  0.,  0.],
       [ 0.,  2.,  0.,  0., -1.,  0.,  0., -1.,  0.],
       [ 0.,  0.,  2.,  0.,  0., -1.,  0.,  0., -1.],
       [-1.,  0.,  0.,  2.,  0.,  0., -1.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  2.,  0.,  0., -1.,  0.],
       [ 0.,  0., -1.,  0.,  0.,  2.,  0.,  0., -1.],
       [-1.,  0.,  0., -1.,  0.,  0.,  2.,  0.,  0.],
       [ 0., -1.,  0.,  0., -1.,  0.,  0.,  2.,  0.],
       [ 0.,  0., -1.,  0.,  0., -1.,  0.,  0.,  2.]])

In [27]:
a_mat_generator(0, 0  , 3, 3)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [3]:
import itertools
sets = {i for i in range(10)}
combinations_2 = list(itertools.combinations(sets, 2))

In [8]:
len(combinations_2)

45

In [31]:
combination = combinations_2.pop()

In [32]:
combination

(8, 9)

In [4]:
from snv import *
a_mat_generator(1,2, 3, 3)

array([[ 0.,  0.,  0.,  1.,  0.,  0., -1., -0., -0.],
       [ 0.,  0.,  0.,  0.,  1.,  0., -0., -1., -0.],
       [ 0.,  0.,  0.,  0.,  0.,  1., -0., -0., -1.]])

In [11]:
a_trans_a_mat_generator(5,4)

array([[ 4.,  0.,  0.,  0., -1.,  0.,  0.,  0., -1.,  0.,  0.,  0., -1.,
         0.,  0.,  0., -1.,  0.,  0.,  0.],
       [ 0.,  4.,  0.,  0.,  0., -1.,  0.,  0.,  0., -1.,  0.,  0.,  0.,
        -1.,  0.,  0.,  0., -1.,  0.,  0.],
       [ 0.,  0.,  4.,  0.,  0.,  0., -1.,  0.,  0.,  0., -1.,  0.,  0.,
         0., -1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  4.,  0.,  0.,  0., -1.,  0.,  0.,  0., -1.,  0.,
         0.,  0., -1.,  0.,  0.,  0., -1.],
       [-1.,  0.,  0.,  0.,  4.,  0.,  0.,  0., -1.,  0.,  0.,  0., -1.,
         0.,  0.,  0., -1.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  0.,  4.,  0.,  0.,  0., -1.,  0.,  0.,  0.,
        -1.,  0.,  0.,  0., -1.,  0.,  0.],
       [ 0.,  0., -1.,  0.,  0.,  0.,  4.,  0.,  0.,  0., -1.,  0.,  0.,
         0., -1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  0.,  0.,  4.,  0.,  0.,  0., -1.,  0.,
         0.,  0., -1.,  0.,  0.,  0., -1.],
       [-1.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  4.,  0.,  0.,  

In [8]:
from scipy.optimize import minimize, rosen, rosen_der
x0 = [1.3, 0.7, 0.8, 1.9, 1.2]
res = minimize(rosen, x0, method='BFGS', jac=rosen_der,
               options={'gtol': 1e-6, 'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 26
         Function evaluations: 31
         Gradient evaluations: 31


In [10]:
rosen

<function scipy.optimize._optimize.rosen(x)>

In [4]:
class aa:
    def __init__(self, lst):
        self.val = lst
    
    def __getitem__(self, index):
        return self.val[index]
    
a = aa([1, 2, 3])

In [6]:
a[0:2]

[1, 2]

In [10]:
sets = {i for i in range(3)}
combinations_2 = list(itertools.combinations(sets, 2))


In [16]:
a, b = combinations_2[1]

In [19]:
b

2